# IP Address Lookup (BMC Discovery)

This notebook reproduces the DisMAL `ipaddr` report for a specific IP address.
It reads connection details from `config.yaml`, queries the appliance for matching devices, discovery access/session results,
and enriches with credential details before exporting a CSV.

## Parameters

In [ ]:
IPADDR = ''  # e.g., '10.0.0.15'


## Requirements

We use `tideway` from pip (remote), plus `pandas` and `PyYAML`.
Uncomment the following to install in your environment if needed.

In [ ]:
# %pip install -q tideway pandas pyyaml

import os, sys
from pathlib import Path
from typing import Any, Dict, List
import pandas as pd
import yaml


## Select Appliance (optional)

If your `config.yaml` defines multiple appliances under the `appliances:` list,
set `APPLIANCE_NAME` to one of their names (e.g., 'prod' or 'dev') or use the index.
Defaults to the first appliance if neither is set.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection


## Configuration (from config.yaml)

Reads settings from `../config.yaml` including target, token/token_file,
API version, and SSL verification preference.
Saves the CSV to `../output_<target>/ipaddr_<ip>.csv`.

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

# Appliance selection
apps = cfg.get('appliances') or []
selected = None
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')

# Token handling: inline token or token file
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

# Version and SSL
API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

# Output path
sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Base Host      :', target)
print('API Version    :', API_VERSION)
print('Verify SSL     :', VERIFY_SSL)
print('Output folder  :', output_dir)
print('Token set      :', bool(token))

# Import tideway from pip; install if needed (respects PIP_INDEX_URL/PIP_EXTRA_INDEX_URL)
try:
    import tideway  # type: ignore
except Exception:
    import subprocess
    print('Installing tideway via pip...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tideway'])
    import tideway  # retry

API_VERSION_NUM = API_VERSION.lstrip('v')
app = tideway.appliance(target, token, api_version=API_VERSION_NUM, ssl_verify=VERIFY_SSL)
twsearch = app.data()
twcreds = app.credentials()

# Quick probe (optional)
try:
    about = app.api_about
    print('Appliance reachable:', about.status_code)
except Exception as e:
    print('Warning: failed to contact appliance /api/about:', e)


## Helpers (normalization, credentials)

In [ ]:
def list_table_to_json(table_like: List[List[Any]]) -> List[Dict[str, Any]]:
    if not table_like or not isinstance(table_like, list):
        return []
    if not table_like or not isinstance(table_like[0], list):
        return []
    headers = table_like[0]
    rows = table_like[1:]
    out: List[Dict[str, Any]] = []
    for r in rows:
        try:
            out.append(dict(zip(headers, r)))
        except Exception:
            continue
    return out

def to_rows(payload: Any) -> List[Dict[str, Any]]:
    if isinstance(payload, list):
        if payload and isinstance(payload[0], list):
            return list_table_to_json(payload)
        if payload and isinstance(payload[0], dict):
            return payload
        return []
    if hasattr(payload, 'json'):
        try:
            js = payload.json()
        except Exception:
            return []
        if isinstance(js, list):
            if js and isinstance(js[0], list):
                return list_table_to_json(js)
            if js and isinstance(js[0], dict):
                return js
        if isinstance(js, dict) and 'results' in js and 'headings' in js:
            table_like = [js['headings']] + list(js.get('results') or [])
            return list_table_to_json(table_like)
        return []
    if isinstance(payload, dict) and 'results' in payload and 'headings' in payload:
        table_like = [payload['headings']] + list(payload.get('results') or [])
        return list_table_to_json(table_like)
    return []

def get_credential_map(twcreds_handle) -> Dict[str, Dict[str, Any]]:
    resp = twcreds_handle.get_vault_credentials
    items = to_rows(resp) if hasattr(resp, 'json') else (resp or [])
    if hasattr(resp, 'json'):
        try:
            items = resp.json()
        except Exception:
            items = []
    mapping: Dict[str, Dict[str, Any]] = {}
    if isinstance(items, list):
        for c in items:
            if not isinstance(c, dict):
                continue
            uuid = str(c.get('uuid') or '').split('/')[-1]
            if not uuid:
                continue
            mapping[uuid] = {
                'label': c.get('label'),
                'username': c.get('username')
                            or c.get('snmp.v3.securityname')
                            or c.get('aws.access_key_id')
                            or c.get('azure.application_id'),
                'enabled': c.get('enabled'),
            }
    return mapping


## Build queries

In [ ]:
if not IPADDR:
    raise ValueError('Please set IPADDR to a valid IP address')

devices_q = f'''
                    search flags(no_segment) Host, NetworkDevice, Printer, SNMPManagedDevice, StorageDevice, ManagementController
                    where '{IPADDR}' in __all_ip_addrs
                    show
                    name as 'Name',
                    os as 'OS',
                    kind(#) as 'Nodekind'
                    processwith unique()
                '''

accesses_q = f'''
                    search DiscoveryAccess where endpoint = '{IPADDR}'
                    show
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.hostname as 'DeviceInfo.hostname',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.device_type as 'DeviceInfo.device_type',
                    inferred_kind as 'nodekind',
                    (#DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_credential
                        or #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_slave) as 'DeviceInfo.last_credential',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.last_access_method as 'DeviceInfo.last_access_method',
                    #DiscoveryAccess:DiscoveryAccessResult:DiscoveryResult:DeviceInfo.method_success as 'DeviceInfo.method_success',
                    'Credential ID Retrieved from DeviceInfo' as 'message'
                    process with unique()
                '''

sessions_q = f'''
                    search DiscoveryAccess where endpoint = '{IPADDR}'
                    traverse DiscoveryAccess:Metadata:Detail:SessionResult
                    show
                    session_type as 'session_type',
                    credential as 'credential',
                    success as 'success',
                    message as 'message',
                    kind(#) as 'nodekind'
                    processwith unique()
                '''

dropped_q = """
                    search DroppedEndpoints
                    show explode endpoints as 'Endpoint'
                """


## Run and assemble

In [ ]:
dev_rows = to_rows(twsearch.search({'query': devices_q}, format='object', limit=500))
acc_rows = to_rows(twsearch.search({'query': accesses_q}, format='object', limit=500))
ses_rows = to_rows(twsearch.search({'query': sessions_q}, format='object', limit=5000))
drop_rows = to_rows(twsearch.search({'query': dropped_q}, format='object', limit=5000))

devices_found = sorted(list({r.get('Name') for r in dev_rows if r.get('Name')} | {r.get('DeviceInfo.hostname') for r in acc_rows if r.get('DeviceInfo.hostname')}))
print(f'IP Address Lookup: {IPADDR}')
print('Devices Found:', devices_found if devices_found else '[none]')
if any((r.get('Endpoint') == IPADDR) for r in drop_rows):
    print('Note: IP is present in DroppedEndpoints')

# Credential enrichment
cred_map = get_credential_map(twcreds)
rows_out = []
for r in (ses_rows or acc_rows):
    uuid = r.get('credential') or r.get('DeviceInfo.last_credential')
    cm = cred_map.get(str(uuid).split('/')[-1]) if uuid else None
    rows_out.append({
        'Session Type': r.get('session_type') or r.get('DeviceInfo.last_access_method'),
        'Credential': (cm or {}).get('label'),
        'Credential ID': uuid,
        'Credential Login': (cm or {}).get('username'),
        'Status': ('Enabled' if (cm or {}).get('enabled') else ('Disabled' if cm and cm.get('enabled') is False else None)),
        'Message': r.get('message'),
        'Successful': r.get('success') if 'success' in r else r.get('DeviceInfo.method_success'),
    })
df = pd.DataFrame(rows_out)
print(f'Total session rows: {len(df)}')
display(df.head(20)) if not df.empty else print('No session results.')


## Save CSV

In [ ]:
df_out = df.copy()
df_out.insert(0, 'Discovery Instance', target)
other_cols = [c for c in ['Session Type','Credential','Credential ID','Credential Login','Status','Message','Successful'] if c in df_out.columns]
df_out = df_out[['Discovery Instance'] + other_cols]

safe_ip = str(IPADDR).replace(':','_').replace('/','_')
OUTPUT_CSV = str(output_dir / f'ipaddr_{safe_ip}.csv')
df_out.to_csv(OUTPUT_CSV, index=False)
print(f'Saved to {OUTPUT_CSV}')
